In [1]:
# 决策树和k近邻都可以用来分类，但是k近邻无法给出数据的内在含义，而决策树可以
# 决策树可能产生过度匹配的问题
# 我们构造决策树，首先要找到决定性作用的特征
# 那么就需要评估每个特征
# 就是不同的往下划分数据子集，直到所有有相同类型的数据都在一个子集中


In [2]:
# 划分数据可以二分法，也可以其他的什么分成四块这样子

In [3]:
# 划分数据集的最大原则就是让无序的数据变得更加有序，也就是信息增益
# 熵就是信息的期望值
# 待分类的事务可能划分在多个分类中，分类x的信息定义：-log2乘上x的概率
# 香农熵：就是计算所有类别所有可能包含的信息期望值

In [4]:
from math import log

def calcShannonEnt(dataSet):
    numEntries = len(dataSet)  # 计算数据集中的实例总数
    labelCounts = {}
    # 以下五行为所有可能分类创建字典
    for featVec in dataSet:
        currentLabel = featVec[-1]  #提取最后一项做为标签
        if currentLabel not in labelCounts.keys():
            # 如果当前键值不存在，则扩展字典并将当前键值加入字典
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1  # 书中有错
    # 0:{"yes":1} 1:{"yes":2}  2:{"no":1} 3:{"no":2} 4:{"no":3}
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key]) / numEntries  # 计算概率
        # 以2为底求对数
        shannonEnt -= prob * log(prob,2) # 递减求和得熵
    return shannonEnt

In [5]:
# 写一个数据集，判断是不是鱼
def createDataSet():
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    labels = ['no surfacing', 'flippers']
    # 不浮出水面是否可以生存，是否有脚蹼
    return dataSet,labels

In [6]:
myData, labels = createDataSet()
calcShannonEnt(myData)
# 熵越高，则混合的数据也越多，可以试试在数据集中添加更多的分类

0.9709505944546686

In [8]:
myData[0][-1] = 'maybe'
# 这里我们增加第三个名为maybe的分类，测试熵的变化
calcShannonEnt(myData)

1.3709505944546687

In [9]:
def splitDataSet(dataSet,axis,value): # 三个输入参数：待划分的数据集、划分数据集的特征、特征的返回值
    # 创建新的list对象
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:  # dataSet[0]=0时执行以下操作
            # 以下三行抽取
            reducedFeatVec = featVec[:axis]   # featVec[:0]= [],即生成一个空列表
            reducedFeatVec.extend(featVec[axis + 1:]) # 添加index==1及后的元素 : 0/1/2 跳过,3:1,4:1
            retDataSet.append(reducedFeatVec) #整体作为元素添加 3:[[1,"no"]] , 4:[[1,"no"],[1,"no"]]
    return retDataSet
# 这个函数是拿来划分数据集的，来判断是否正确的划分了数据集，判断按照哪个特征划分数据集是最好的划分方式

In [11]:
splitDataSet(myData, 0 ,1)

[[1, 'maybe'], [1, 'yes'], [0, 'no']]

In [12]:
splitDataSet(myData, 0 ,0)

[[1, 'no'], [1, 'no']]

In [15]:
# 接下来我们遍历整个数据集，循环计算

# 选择最好的数据集划分方式
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1 # 去掉标签项
    baseEntropy = calcShannonEnt(dataSet) # 计算熵
    bestInfoGain = 0.0
    bestFeature = -1
    for i in range(numFeatures):
        # 以下两行创建唯一的分类标签列表
        featList = [example[i] for example in dataSet] # i=0:[1,1,1,0,0]  i=1:[1,1,0,1,1]
        uniqueVals = set(featList)  # i=0:{0,1}  i=1:{0,1}
        newEntropy = 0.0
        # 以下五行计算每种划分方式的信息熵
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            print(subDataSet)
            # i=0:{(0,0),(0,1)} 返回:[[1, 'no'], [1, 'no']]      [[1,"yes"],[1,"yes"],[0,"no"]]
            # i=1:{(1,0),(1,1)} 返回:[[0,"no"]]       [[1,"yes"],[1,"yes"],[1,"no"],[1,"no"]]
            prob = len(subDataSet)/float(len(dataSet))
            # i=0:{(0,0),(0,1)} 返回:2/5 3/5
            # i=1:{(1,0),(1,1)} 返回:1/5 4/5
            newEntropy += prob * calcShannonEnt(subDataSet)  # 注意这里是subDataSet 不是 dataSet
        print("当i={}时得到的熵为".format(i),newEntropy)
        
        infoGain = baseEntropy - newEntropy # 信息增益
        if (infoGain > bestInfoGain):
            # 计算最好的信息增益
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature
chooseBestFeatureToSplit(myData)
# 熵越小越好，越不混乱

[[1, 'no'], [1, 'no']]
[[1, 'maybe'], [1, 'yes'], [0, 'no']]
当i=0时得到的熵为 0.9509775004326936
[[1, 'no']]
[[1, 'maybe'], [1, 'yes'], [0, 'no'], [0, 'no']]
当i=1时得到的熵为 1.2000000000000002


0

In [ ]:
# 得到原始的数据集，基于最好的属性划分数据集，递归一次次划分数据
